# Foursquare Location Matching

The data presented here comprises over one-and-a-half million place entries for hundreds of thousands of commercial Points-of-Interest (POIs) around the globe. 

My task is to determine which place entries describe the same point-of-interest. Though the data entries may represent or resemble entries for real places, they may also contain artificial information or additional noise.

## Libraries installing and dataset downloading

In [2]:
%%capture

! pip install kaggle
! pip install numpy
! pip install pandas
! pip install sklearn

In [3]:
# Flag to force to reload dataset
RELOAD = False

In [4]:
import os

# import Kaggle API to load dataset
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

# initialize Kaggle API
api = KaggleApi()
api.authenticate()

# download dataset from Kaggle to data folder
data_path = 'data'
api.competition_download_files('foursquare-location-matching', data_path, force=RELOAD, quiet=False)
# save filename: !ATTENTION! : it may not be wroking if many files are in folders
# then just name it manually 
dataset_file_name = os.listdir(data_path)[0]

foursquare-location-matching.zip: Skipping, found more recently modified local copy (use --force to force download)


### Data description and loading

The data presented here comprises over one-and-a-half million place entries for hundreds of thousands of commercial Points-of-Interest (POIs) around the globe. Your task is to determine which place entries describe the same point-of-interest. Though the data entries may represent or resemble entries for real places, they may also contain artificial information or additional noise.

#### Training Data

* *train.csv* - The training set, comprising eleven attribute fields for over one million place entries, together with:
    * `id` - A unique identifier for each entry.
    * `point_of_interest` - An identifier for the POI the entry represents. There may be one or many entries describing the same POI. Two entries "match" when they describe a common POI.
* *pairs.csv* - A pregenerated set of pairs of place entries from train.csv designed to improve detection of matches. You may wish to generate additional pairs to improve your model's ability to discriminate POIs.
    * `match` - Whether (`True` or `False`) the pair of entries describes a common POI.

#### Example Test Data

To help you author submission code, we include a few example instances selected from the test set. When you submit your notebook for scoring, this example data will be replaced by the actual test data. The actual test set has approximately 600,000 place entries with POIs that are distinct from the POIs in the training set.

* *test.csv* - A set of place entries with their recorded attribute fields, similar to the training set.
* *sample_submission.csv* - A sample submission file in the correct format.
    * `id` - The unique identifier for a place entry, one for each entry in the test set.
    * `matches` - A space delimited list of IDs for entries in the test set matching the given ID. Place entries always self-match.

In [5]:
# import libraries to work with paths and to read zipped file, as was downloaded from Kaggle
from zipfile import ZipFile
# import pandas for EDA
import pandas as pd

# Read train dataset (train.csv) to pandas DataFrame named df: it will be used for analysis
df = pd.read_csv(ZipFile(os.path.join(data_path, dataset_file_name)).open('train.csv'))

df_pairs = pd.read_csv(ZipFile(os.path.join(data_path, dataset_file_name)).open('pairs.csv'))

# Read test dataset (test.csv), to pandas DataFrame named df_validation. It will be used only to generate final predictions, which will be submitted
df_validation = pd.read_csv(ZipFile(os.path.join(data_path, dataset_file_name)).open('test.csv'))
# finally, we will download example of submission (there are no correct predictions there, it is just an example)
df_subm_example = pd.read_csv(ZipFile(os.path.join(data_path, dataset_file_name)).open('sample_submission.csv'))

In [6]:
# Check, that all dataframes are loaded and have correct shapes
print(f'Shape of df: {str(df.shape)}')
print(f'Shape of df_pairs: {str(df_pairs.shape)}')
print(f'Shape of df_validation: {str(df_validation.shape)}')
print(f'Shape of df_subm_example: {str(df_subm_example.shape)}')

Shape of df: (1138812, 13)
Shape of df_pairs: (578907, 25)
Shape of df_validation: (5, 12)
Shape of df_subm_example: (5, 2)


## Exploratory data analysis

First, let's take a look on `df`, analyse it's structure and data

In [7]:
df.head()

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e
1,E_000002eae2a589,Carioca Manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,BR,NaN,NaN,Brazilian Restaurants,P_d82910d8382a83
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,TH,NaN,NaN,Salons / Barbershops,P_b1066599e78477
3,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops,P_b2ed86905a4cd3
4,E_00001d92066153,Restaurante Casa Cofiño,43.338196,-4.326821,NaN,Caviedes,Cantabria,NaN,ES,NaN,NaN,Spanish Restaurants,P_809a884d4407fb


In [8]:
print(f'Number of records in df: {df.shape[0]}')
print(f'Number of columns in df: {df.shape[1]}')

Number of records in df: 1138812
Number of columns in df: 13


In [9]:
print(f'Names of columns in df: {list(df.columns)}')

Names of columns in df: ['id', 'name', 'latitude', 'longitude', 'address', 'city', 'state', 'zip', 'country', 'url', 'phone', 'categories', 'point_of_interest']


DataFrame `df` contains entries of POIs. 
Let's go through the columns to describe them

First, let's create helper function `col_describe` to use it for different columns

In [10]:
def col_describe(column_name, df=df):
    t = df[column_name].dtype
    print(f"Type of `{column_name}` column in `df` is: {t}")
    if (t == object):
        print("Object in pandas means string")
    print(f"Number of NaNs in `{column_name}` column: {df.isna()[column_name].sum()}")
    n = len(df[column_name])
    nu = df[column_name].nunique()
    print(f"Total amount of records (column `{column_name}`) is {n} and number of unique values is {nu}")
    print(f"{nu / n * 100}% of the values in `{column_name}` column are unique")


Column `id` is a unique identifier of the entry

In [11]:
col_describe('id')

Type of `id` column in `df` is: object
Object in pandas means string
Number of NaNs in `id` column: 0
Total amount of records (column `id`) is 1138812 and number of unique values is 1138812
100.0% of the values in `id` column are unique


Column `name` is the name of POI entry

In [12]:
col_describe('name')

Type of `name` column in `df` is: object
Object in pandas means string
Number of NaNs in `name` column: 1
Total amount of records (column `name`) is 1138812 and number of unique values is 842086
73.9442506752651% of the values in `name` column are unique


Columns `latitude` and `longitude` are geographical coordinates of the reported location

In [13]:
col_describe('latitude')
print('\n')
col_describe('longitude')

Type of `latitude` column in `df` is: float64
Number of NaNs in `latitude` column: 0
Total amount of records (column `latitude`) is 1138812 and number of unique values is 1121701
98.49746929256102% of the values in `latitude` column are unique


Type of `longitude` column in `df` is: float64
Number of NaNs in `longitude` column: 0
Total amount of records (column `longitude`) is 1138812 and number of unique values is 1080273
94.85964320713164% of the values in `longitude` column are unique


Columns `address`, `city`, `state`, `zip`, `country` are describing address of the reported location

In [14]:
col_describe('address')
print('\n')
col_describe('city')
print('\n')
col_describe('state')
print('\n')
col_describe('zip')
print('\n')
col_describe('country')

Type of `address` column in `df` is: object
Object in pandas means string
Number of NaNs in `address` column: 396621
Total amount of records (column `address`) is 1138812 and number of unique values is 558154
49.01195280696024% of the values in `address` column are unique


Type of `city` column in `df` is: object
Object in pandas means string
Number of NaNs in `city` column: 299189
Total amount of records (column `city`) is 1138812 and number of unique values is 68105
5.980354966403585% of the values in `city` column are unique


Type of `state` column in `df` is: object
Object in pandas means string
Number of NaNs in `state` column: 420586
Total amount of records (column `state`) is 1138812 and number of unique values is 17596
1.5451189485182804% of the values in `state` column are unique


Type of `zip` column in `df` is: object
Object in pandas means string
Number of NaNs in `zip` column: 595426
Total amount of records (column `zip`) is 1138812 and number of unique values is 93329


Columns `url`, `phone` of the POI

In [15]:
col_describe('url')
print('\n')
col_describe('phone')

Type of `url` column in `df` is: object
Object in pandas means string
Number of NaNs in `url` column: 871088
Total amount of records (column `url`) is 1138812 and number of unique values is 171222
15.035141884700899% of the values in `url` column are unique


Type of `phone` column in `df` is: object
Object in pandas means string
Number of NaNs in `phone` column: 795957
Total amount of records (column `phone`) is 1138812 and number of unique values is 293454
25.768432366360734% of the values in `phone` column are unique


Column `categories` describes category of the entry (Bar, Shop etc)

In [26]:
col_describe('categories')

Type of `categories` column in `df` is: object
Object in pandas means string
Number of NaNs in `categories` column: 98307
Total amount of records (column `categories`) is 1138812 and number of unique values is 52578
4.616916576221536% of the values in `categories` column are unique


Column `point_of_interest`: an identifier for the POI the entry represents. There may be one or many entries describing the same POI. Two entries "match" when they describe a common POI.

In [16]:
col_describe('point_of_interest')

Type of `point_of_interest` column in `df` is: object
Object in pandas means string
Number of NaNs in `point_of_interest` column: 0
Total amount of records (column `point_of_interest`) is 1138812 and number of unique values is 739972
64.97753799573591% of the values in `point_of_interest` column are unique


Task can be formulated in a way, that we need to create a model, which is able to group entries together, which have the same `point_of_interest`.

Let's take a look on some entries with the most popular POI:

In [23]:
df.groupby('point_of_interest')['id'].nunique().sort_values(ascending=False)

point_of_interest
P_fb339198a31db3    332
P_3a3fbd26e16269    199
P_f70a0977377171    166
P_9290e406eb58ac    145
P_fbe61733fcdebe    132
                   ... 
P_6c88990990db38      1
P_6c884940447aa7      1
P_6c8847f66224e2      1
P_6c883c00a983b6      1
P_7ff3b6cf9ac32e      1
Name: id, Length: 739972, dtype: int64

We see POI P_fb339198a31db3 332 times


Let's see on five entries which corresponds to this POI

In [46]:
df[df['point_of_interest'] == 'P_fb339198a31db3'].head()

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
10820,E_02761e40f68230,Sukarno Hatta airport,-7.329110,108.213608,NaN,NaN,NaN,NaN,ID,NaN,NaN,Airport Terminals,P_fb339198a31db3
14686,E_0351377fc6b71d,Bandara Soekarno Hatta,-6.143365,106.729291,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,P_fb339198a31db3
19520,E_046132a06d8bc5,Bandar Djakarta,-6.117818,106.855710,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,P_fb339198a31db3
21192,E_04c39deab3aba2,"Bandara Soekarnao Hatta...^^,",-6.234635,106.874453,NaN,NaN,NaN,NaN,ID,NaN,NaN,Airport Terminals,P_fb339198a31db3
21433,E_04d15c990bfd9e,Bandara Soekarno Hatta,1.541611,98.915245,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,P_fb339198a31db3


First, we can see, that all entries in the same country: Indonesia. No addresses, phone numbers, URLs etc. Geographical coordinates are close to each other, but not very presise. Even one outlier (with `id` E_04d15c990bfd9e)

Let's check, if we have entries for the same POI with values in address and other columns:

In [44]:
df.loc[(df['point_of_interest'] == 'P_fb339198a31db3') & (~df['address'].isna()) & (~df['phone'].isna())]

,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
271614,E_3cfcdf84257fd2,Bandara Soekarno Hatta,-8.504281,117.425167,Jakarta,Jakarta,Jakarta,84353,ID,NaN,+6237121006,Lounges,P_fb339198a31db3
519362,E_74ba66c8496334,Soekarno-Hatta International Airport (CGK),-6.125748,106.656475,Jalan Raya Bandara,Tangerang,Banten,15126,ID,http://soekarnohatta-airport.co.id,215507300,Airports,P_fb339198a31db3
708256,E_9f1603ea1e92bb,Jakarta International Soekarno Hatta Airport B...,-6.126090,106.657441,Tangerang - Banten,Jakarta,Jakarta,NaN,ID,NaN,215506823,Airport Terminals,P_fb339198a31db3
1030082,E_e76f85ed8f35fc,Soekarno Hatta International Airport Of Jakarta,-8.742641,115.167138,Jakarta International Airport Soekarno - Hatta...,Tangerang,Banten,19101,ID,NaN,215507300,Airport Terminals,P_fb339198a31db3
1073029,E_f127bc8fcc9a60,Soekarno Hatta International Airport (CGK),-6.125748,106.656475,Jakarta,Jakarta,Jakarta,70111,ID,NaN,85750044700,Airport Terminals,P_fb339198a31db3


We can see, that `address` field can contain uncomlete information and `phone` can be in different formats, can be different (several phones in the company?). But similarity in the phone numbers (first *n* digits are the same) can mean that it is the same company.

Let's take a look on *pairs.csv* file, which contains pairs of entries which corresponds to the same POI:

In [39]:
df_pairs.head()

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,...,3.635206,NaN,NaN,NaN,NaN,BE,NaN,NaN,Bars,True
1,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,...,27.845560,batı aydın,aydın,NaN,67500,TR,NaN,NaN,Electronics Stores,False
2,E_000023d8f4be44,Island Spa,14.518970,121.018702,"5th Flr, Newport Mall, Resorts World Manila",Pasay City,Metro Manila,NaN,PH,NaN,...,121.019827,NaN,NaN,NaN,NaN,PH,NaN,NaN,Spas,True
3,E_00007dcd2bb53f,TOGO'S Sandwiches,38.257797,-122.064599,"1380 Holiday Ln., Ste. B",Fairfield,CA,94534,US,https://locations.togos.com/ll/US/CA/Fairfield...,...,-122.064606,NaN,Fairfield,CA,NaN,US,NaN,NaN,Sandwich Places,True
4,E_0000c362229d93,Coffee Cat,7.082218,125.610244,F. Torres St.,Davao City,Davao Region,8000,PH,NaN,...,125.611457,E. Jacinto Extension,Davao City,Davao Region,8000,PH,NaN,NaN,"Coffee Shops, Cafés, Dessert Shops",False


It contains 25 columns (13 in original dataset, minus `point_of_interest`, multiply by 2, because we compare two entries, and plus one for binary column `match`)

Let's find the same entries, which we were looking before (Jakarta Airport):

In [47]:
# Check, which entries from df (by id), which correspond to POI P_fb339198a31db3 present in df_pairs
# save filtered df
df_one_POI = df[df['point_of_interest'] == 'P_fb339198a31db3']
# and check size
df_one_POI.shape

(332, 13)

In [60]:
# Save id's to another variable
id_one_POI = list(df_one_POI['id'])

# Create boolean list for filtering
is_in_list = df_pairs['id_1'].isin(id_one_POI) & df_pairs['id_2'].isin(id_one_POI)

# and filter df_pairs to have id_1 or id_2 in the id_one_POI variable
df_pairs[is_in_list].head()


,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
264690,E_74ba66c8496334,Soekarno-Hatta International Airport (CGK),-6.125748,106.656475,Jalan Raya Bandara,Tangerang,Banten,15126,ID,http://soekarnohatta-airport.co.id,...,108.213608,NaN,NaN,NaN,NaN,ID,NaN,NaN,Airport Terminals,True
264691,E_74ba66c8496334,Soekarno-Hatta International Airport (CGK),-6.125748,106.656475,Jalan Raya Bandara,Tangerang,Banten,15126,ID,http://soekarnohatta-airport.co.id,...,106.729291,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,True
264692,E_74ba66c8496334,Soekarno-Hatta International Airport (CGK),-6.125748,106.656475,Jalan Raya Bandara,Tangerang,Banten,15126,ID,http://soekarnohatta-airport.co.id,...,106.855710,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,True
264693,E_74ba66c8496334,Soekarno-Hatta International Airport (CGK),-6.125748,106.656475,Jalan Raya Bandara,Tangerang,Banten,15126,ID,http://soekarnohatta-airport.co.id,...,106.874453,NaN,NaN,NaN,NaN,ID,NaN,NaN,Airport Terminals,True
264694,E_74ba66c8496334,Soekarno-Hatta International Airport (CGK),-6.125748,106.656475,Jalan Raya Bandara,Tangerang,Banten,15126,ID,http://soekarnohatta-airport.co.id,...,98.915245,NaN,NaN,NaN,NaN,ID,NaN,NaN,NaN,True


In [41]:
df_pairs.loc[(df_pairs['id_1'] == 'E_3cfcdf84257fd2') & (df_pairs['id_2'] == 'E_9f1603ea1e92bb')]

,id_1,name_1,latitude_1,longitude_1,address_1,city_1,state_1,zip_1,country_1,url_1,...,longitude_2,address_2,city_2,state_2,zip_2,country_2,url_2,phone_2,categories_2,match
